In [1]:
!apt-get update && apt-get install -y nodejs npm g++
!pip install python-dotenv openai gradio

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,471 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,574 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/ma

In [ ]:
# Multi-Language Code Converter (Python, C++, JavaScript) 


import os
import io
import sys
import tempfile
import subprocess
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import platform
import re

# set your OpenRouter API key here
OPENROUTER_API_KEY = ""

# Initialize OpenAI client with OpenRouter endpoint
openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)


DEEPSEEK_MODEL = "deepseek/deepseek-chat-v3-0324:free"

def write_output(code, file_name="optimized.cpp"):
    """Write the generated code to a file."""
    with open(file_name, "w") as f:
        f.write(code)

def system_message_for(language, source_language="python"):
    """Create a system message tailored for the requested language conversion."""
    array_instruction = (
        " When converting array code, ensure that multi-dimensional arrays, slicing, and all array methods are mapped to the closest equivalent in the target language. "
        "If the target language does not support a feature, provide a workaround."
    )

    if language == "javascript":
        if source_language == "python":
            return (
                "You are an assistant that converts Python code to JavaScript (Node.js). "
                "Respond ONLY with valid JavaScript code wrapped in ```javascript code blocks. "
                "Convert Python's print() to console.log(). Use appropriate JavaScript syntax for loops, functions, etc. "
                "Handle Python's indentation-based blocks with JavaScript's brace-based blocks. "
                "Convert Python imports to Node.js require() statements when needed. "
                "Use const/let appropriately and follow modern JavaScript practices. "
                "Ensure all variables are properly declared with const, let, or var. "
                "Do NOT include any explanations outside the code block." + array_instruction
            )
        elif source_language == "cpp":
            return (
                "You are an assistant that converts C++ code to JavaScript (Node.js). "
                "Respond ONLY with valid JavaScript code wrapped in ```javascript code blocks. "
                "Convert std::cout to console.log(). Convert C++ syntax to JavaScript equivalents. "
                "Handle C++ types by using appropriate JavaScript data types. "
                "Convert C++ loops and control structures to JavaScript syntax. "
                "Remove C++ headers and main() function wrapper, keep only the core logic. "
                "Ensure all variables are properly declared with const, let, or var. "
                "Convert C++ #include statements and using namespace to appropriate JavaScript equivalents. "
                "Do NOT include any explanations outside the code block." + array_instruction
            )
        else:
            return (
                f"You are an assistant that converts {source_language.title()} code to JavaScript. "
                f"Respond ONLY with valid JavaScript code wrapped in ```javascript code blocks. "
                f"Use Node.js compatible syntax and ensure identical output. "
                f"Ensure all variables are properly declared with const, let, or var. "
                f"Do NOT include any explanations outside the code block." + array_instruction
            )

    elif language == "python":
        if source_language == "javascript":
            return (
                "You are an assistant that converts JavaScript code to Python. "
                "Respond ONLY with valid Python code wrapped in ```python code blocks. "
                "Convert console.log() to print(). Convert JavaScript functions to Python functions. "
                "Use Python's indentation-based syntax instead of braces. "
                "Convert let/const/var declarations to Python variables (no declaration needed). "
                "Handle JavaScript's camelCase with Python's snake_case when appropriate. "
                "Do NOT include any explanations outside the code block." + array_instruction
            )
        elif source_language == "cpp":
            return (
                "You are an assistant that converts C++ code to Python. "
                "Respond ONLY with valid Python code wrapped in ```python code blocks. "
                "Convert std::cout to print(). Extract the main logic from C++ structure. "
                "Convert C++ static typing to Python's dynamic typing. "
                "Use Python's idiomatic syntax and data structures. "
                "Remove C++ headers and main() function, keep only the core logic. "
                "Do NOT include any explanations outside the code block." + array_instruction
            )
        else:
            return (
                f"You are an assistant that converts {source_language.title()} code to Python. "
                f"Respond ONLY with valid Python code wrapped in ```python code blocks. "
                f"Use idiomatic Python syntax and ensure identical output. "
                f"Do NOT include any explanations outside the code block." + array_instruction
            )

    elif language == "cpp":
        if source_language == "javascript":
            return (
                "You are an assistant that converts JavaScript code to C++. "
                "Respond ONLY with valid C++ code wrapped in ```cpp code blocks. "
                "Convert console.log() to std::cout. Include necessary headers like <iostream>, <vector>, etc. "
                "Convert JavaScript's dynamic typing to appropriate C++ types (int, double, string, etc.). "
                "Add proper main() function and handle memory management if needed. "
                "Use modern C++ practices and ensure the code compiles with g++. "
                "Do NOT include any explanations outside the code block." + array_instruction
            )
        elif source_language == "python":
            return (
                "You are an assistant that converts Python code to C++. "
                "Respond ONLY with valid C++ code wrapped in ```cpp code blocks. "
                "Convert print() to std::cout. Include all necessary headers. "
                "Convert Python's dynamic typing to appropriate C++ types. "
                "Add proper main() function and use modern C++ syntax. "
                "Do NOT include any explanations outside the code block." + array_instruction
            )
        else:
            return (
                f"You are an assistant that converts {source_language.title()} code to C++. "
                f"Respond ONLY with valid C++ code wrapped in ```cpp code blocks. "
                f"Include all necessary headers and use appropriate C++ syntax. "
                f"Ensure the code compiles and produces identical output. "
                f"Do NOT include any explanations outside the code block." + array_instruction
            )

    else:
        return (
            f"You are an assistant that converts {source_language.title()} code to {language.upper()}. "
            f"Respond ONLY with valid {language.upper()} code wrapped in ```{language} code blocks. "
            f"Ensure the converted code produces identical output. "
            f"Do NOT include any explanations outside the code block." + array_instruction
        )

def user_prompt_for(source_code, source_language, target_language):
    """Generate the user prompt for language conversion."""
    return (
        f"Rewrite this {source_language.title()} code in {target_language.title()} with the fastest possible implementation that produces "
        f"identical output in the least time. Use appropriate syntax for {target_language.title()}.\n\n" + source_code
    )

def messages_for(source_code, target_language, source_language="python"):
    """Generate the messages for DeepSeek."""
    return [
        {"role": "system", "content": system_message_for(target_language, source_language)},
        {"role": "user", "content": user_prompt_for(source_code, source_language, target_language)},
    ]

def extract_code_from_response(response, language):
    """Extract code from AI response, handling various formats."""
    # Remove all code blocks and extract the code inside
    # Try to find a code block with the correct language
    code_block_pattern = rf"```{language}\s*([\s\S]*?)```"
    match = re.search(code_block_pattern, response, re.IGNORECASE)
    if match:
        return match.group(1).strip()

    # Try alternative language names
    alt_names = {
        'javascript': ['js', 'node', 'nodejs'],
        'cpp': ['c++', 'cxx'],
        'python': ['py']
    }

    if language in alt_names:
        for alt_name in alt_names[language]:
            alt_pattern = rf"```{alt_name}\s*([\s\S]*?)```"
            match = re.search(alt_pattern, response, re.IGNORECASE)
            if match:
                return match.group(1).strip()

    # Try to find any code block
    generic_pattern = r"```[\w]*\s*([\s\S]*?)```"
    match = re.search(generic_pattern, response)
    if match:
        return match.group(1).strip()

    # Remove stray triple backticks if present
    code = response.strip()
    code = re.sub(r"^```[\w]*", "", code)
    code = re.sub(r"```$", "", code)

    # Remove any leading/trailing whitespace again
    return code.strip()

def stream_conversion(source_code, target_language="cpp", source_language="python"):
    """Stream the conversion output for the requested language."""
    valid_languages = ["cpp", "javascript", "python"]
    if target_language not in valid_languages:
        raise ValueError(f"Invalid target language specified. Choose from: {valid_languages}")

    try:
        stream = openai.chat.completions.create(
            model=DEEPSEEK_MODEL,
            messages=messages_for(source_code, target_language, source_language),
            stream=True
        )
        reply = ""

        for chunk in stream:
            if chunk.choices[0].delta.content:
                fragment = chunk.choices[0].delta.content
                reply += fragment

                cleaned_code = extract_code_from_response(reply, target_language)
                yield cleaned_code
    except Exception as e:
        yield f"Error: {str(e)}"

def is_valid_python(code):
    """Check if Python code is syntactically valid."""
    import ast
    try:
        ast.parse(code)
        return True, ""
    except SyntaxError as e:
        return False, str(e)

def is_valid_cpp(code):
    """Check if C++ code is syntactically valid."""
    with tempfile.NamedTemporaryFile(suffix='.cpp', delete=False, mode='w') as f:
        f.write(code)
        fname = f.name
    try:
        result = subprocess.run(["g++", "-fsyntax-only", fname], capture_output=True, text=True)
        return result.returncode == 0, result.stderr
    except FileNotFoundError:
        return False, "g++ compiler not found"
    finally:
        if os.path.exists(fname):
            os.remove(fname)

def is_valid_javascript(code):
    """Check if JavaScript code is syntactically valid."""
    with tempfile.NamedTemporaryFile(suffix='.js', delete=False, mode='w') as f:
        f.write(code)
        fname = f.name
    try:
        result = subprocess.run(["node", "--check", fname], capture_output=True, text=True)
        return result.returncode == 0, result.stderr
    except FileNotFoundError:
        return False, "Node.js not found"
    finally:
        if os.path.exists(fname):
            os.remove(fname)

def execute_python(code):
    """Execute Python code safely."""
    is_valid, error = is_valid_python(code)
    if not is_valid:
        return f"Syntax Error: {error}"

    # Create a temporary file for better error handling
    with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
        f.write(code)
        temp_file = f.name

    try:
        # Execute using subprocess for better isolation
        result = subprocess.run(
            [sys.executable, temp_file],
            capture_output=True,
            text=True,
            timeout=30,
            cwd=os.getcwd()
        )

        if result.returncode == 0:
            return result.stdout
        else:
            return f"Runtime Error: {result.stderr}"

    except subprocess.TimeoutExpired:
        return "Error: Python execution timed out (30 seconds)"
    except Exception as e:
        return f"Error during execution: {str(e)}"
    finally:
        if os.path.exists(temp_file):
            os.remove(temp_file)

def execute_cpp(code):
    """Execute C++ code."""
    is_valid, error = is_valid_cpp(code)
    if not is_valid:
        return f"Syntax Error: {error}"

    # Use temporary files for better cleanup
    with tempfile.NamedTemporaryFile(suffix='.cpp', delete=False, mode='w') as f:
        f.write(code)
        cpp_file = f.name

    executable = cpp_file.replace('.cpp', '')

    try:
        # Compile
        compile_cmd = ["g++", "-O3", "-std=c++17", "-o", executable, cpp_file]
        compile_result = subprocess.run(compile_cmd, capture_output=True, text=True)

        if compile_result.returncode != 0:
            return f"Compilation error:\n{compile_result.stderr}"

        # Execute
        run_result = subprocess.run([executable], capture_output=True, text=True, timeout=30)

        if run_result.returncode == 0:
            return run_result.stdout
        else:
            return f"Runtime error:\n{run_result.stderr}"

    except subprocess.TimeoutExpired:
        return "Error: C++ execution timed out (30 seconds)"
    except FileNotFoundError as e:
        return f"Error: {str(e)} - Make sure g++ is installed"
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        # Clean up files
        for file_path in [cpp_file, executable]:
            if os.path.exists(file_path):
                try:
                    os.remove(file_path)
                except:
                    pass

def clean_javascript_code(code):
    """Clean and prepare JavaScript code for execution."""
    # Remove code block markers if present
    code = re.sub(r'^```[\w]*\n?', '', code, flags=re.MULTILINE)
    code = re.sub(r'\n?```$', '', code, flags=re.MULTILINE)

    # Remove any explanatory text that's not JavaScript
    lines = code.split('\n')
    js_lines = []
    in_comment_block = False

    for line in lines:
        stripped = line.strip()

        # Handle multi-line comments
        if '/*' in stripped:
            in_comment_block = True
        if '*/' in stripped:
            in_comment_block = False
            continue
        if in_comment_block:
            continue

        # Skip empty lines and obvious non-JS content
        if not stripped or stripped.startswith('//'):
            js_lines.append(line)
            continue

        # Keep lines that look like JavaScript
        if (any(keyword in stripped for keyword in [
            'console.log', 'function', 'const ', 'let ', 'var ', 'for(', 'for ',
            'while(', 'if(', '=>', 'return', '{', '}', ';'
        ]) or stripped.endswith(';') or stripped.endswith('{') or stripped.endswith('}')):
            js_lines.append(line)
        elif js_lines and not stripped.startswith(('Here', 'This', 'The', 'Note:')):
            # Include lines that seem to be part of the code structure
            js_lines.append(line)

    return '\n'.join(js_lines).strip()

def execute_javascript(code):
    """Execute JavaScript code."""
    # Clean the code first
    cleaned_code = clean_javascript_code(code)

    is_valid, error = is_valid_javascript(cleaned_code)
    if not is_valid:
        return f"Syntax Error: {error}"

    with tempfile.NamedTemporaryFile(suffix='.js', delete=False, mode='w') as f:
        f.write(cleaned_code)
        js_file = f.name

    try:
        # Check if Node.js is available
        node_check = subprocess.run(["node", "--version"], capture_output=True, text=True)
        if node_check.returncode != 0:
            return "Error: Node.js not found. Please install it: !apt-get update && apt-get install -y nodejs npm"

        # Execute the JavaScript file
        run_result = subprocess.run(
            ["node", js_file],
            capture_output=True,
            text=True,
            timeout=30
        )

        if run_result.returncode == 0:
            return run_result.stdout
        else:
            return f"Runtime Error: {run_result.stderr}"

    except subprocess.TimeoutExpired:
        return "Error: JavaScript execution timed out (30 seconds)"
    except FileNotFoundError:
        return "Error: Node.js not found. Please install it: !apt-get update && apt-get install -y nodejs npm"
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        if os.path.exists(js_file):
            os.remove(js_file)

def convert_code(source_code, target_language, source_language="python"):
    """Convert the given source code to target language using DeepSeek."""
    result = ""
    for stream_so_far in stream_conversion(source_code, target_language, source_language):
        result = stream_so_far
    return result

def convert_code_with_retry(source_code, target_language, source_language="python", max_retries=2):
    """Convert code with retry logic for better reliability."""
    for attempt in range(max_retries):
        result = convert_code(source_code, target_language, source_language)

        # Try to execute the code to validate it
        if target_language == "python":
            output = execute_python(result)
        elif target_language == "cpp":
            output = execute_cpp(result)
        elif target_language == "javascript":
            output = execute_javascript(result)
        else:
            output = "Unsupported language"

        # Check if the conversion was successful
        if not any(error_keyword in output for error_keyword in ["Error", "Syntax Error", "Runtime Error"]):
            return result  # Success
        elif attempt < max_retries - 1:  # Not the last attempt
            # Re-prompt LLM with error message
            error_prompt = f"\n\nThe previous attempt failed with this error:\n{output}\nPlease fix the code and ensure it runs correctly."
            source_code += error_prompt

    return result  # Return last attempt even if it failed

# CSS for styling
css = """
.python {background-color: #306998; color: white;}
.cpp {background-color: #f34b7d; color: white;}
.javascript {background-color: #f1e05a; color: #000;}
"""

# Gradio Interface
with gr.Blocks(css=css, title="Multi-Language Code Converter") as ui:
    gr.Markdown("## Multi-Language Code Converter using DeepSeek R1")
    gr.Markdown("Convert between Python, C++, and JavaScript")
    gr.Markdown("**Note:** Make sure to run `!apt-get update && apt-get install -y nodejs npm g++` before using this tool in Colab")

    with gr.Row():
        with gr.Column():
            source_lang = gr.Dropdown(
                ["python", "javascript", "cpp"],
                label="Source Language",
                value="python"
            )
            source_code = gr.Textbox(
                label="Source Code:",
                value="""import time
import math

def calculate_pi(iterations):
    pi_estimate = 0
    for i in range(iterations):
        pi_estimate += ((-1)**i) / (2*i + 1)
    return pi_estimate * 4

start = time.time()
result = calculate_pi(1000000)
end = time.time()

print(f"Pi estimate: {result:.10f}")
print(f"Time taken: {end - start:.6f} seconds")""",
                lines=15
            )

        with gr.Column():
            target_lang = gr.Dropdown(
                ["cpp", "javascript", "python"],
                label="Target Language",
                value="cpp"
            )
            target_code = gr.Textbox(label="Converted Code:", lines=15)

    # Add example code for different languages
    examples = gr.Examples(
        examples=[
            ["python", """# Python example
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print(f"Fibonacci of 10: {fibonacci(10)}")"""],
            ["javascript", """// JavaScript example
function fibonacci(n) {
    if (n <= 1) return n;
    return fibonacci(n-1) + fibonacci(n-2);
}

console.log(`Fibonacci of 10: ${fibonacci(10)}`);"""],
            ["cpp", """// C++ example
#include <iostream>
using namespace std;

int fibonacci(int n) {
    if (n <= 1) return n;
    return fibonacci(n-1) + fibonacci(n-2);
}

int main() {
    cout << "Fibonacci of 10: " << fibonacci(10) << endl;
    return 0;
}"""]
        ],
        inputs=[source_lang, source_code],
        label="Example Code for Different Languages"
    )

    with gr.Row():
        convert_btn = gr.Button("Convert Code", variant="primary")

    with gr.Row():
        run_source = gr.Button("Run Source Code")
        run_target = gr.Button("Run Converted Code")

    with gr.Row():
        source_output = gr.TextArea(label="Source Code Output:")
        target_output = gr.TextArea(label="Converted Code Output:")

    # Event handlers
    def convert_code_wrapper(source, source_lang_val, target_lang_val):
        """Wrapper function for code conversion."""
        if not source.strip():
            return "Error: Please provide source code to convert."
        return convert_code_with_retry(source, target_lang_val, source_lang_val)

    def run_code(code, language):
        """Run code based on the language."""
        if not code.strip():
            return "Error: No code to execute."

        if language == "python":
            return execute_python(code)
        elif language == "cpp":
            return execute_cpp(code)
        elif language == "javascript":
            return execute_javascript(code)
        else:
            return f"Unsupported language: {language}"

    # Connect the UI components
    convert_btn.click(
        convert_code_wrapper,
        inputs=[source_code, source_lang, target_lang],
        outputs=[target_code]
    )

    run_source.click(
        lambda code, lang: run_code(code, lang),
        inputs=[source_code, source_lang],
        outputs=[source_output]
    )

    run_target.click(
        lambda code, lang: run_code(code, lang),
        inputs=[target_code, target_lang],
        outputs=[target_output]
    )

# Launch the interface
if __name__ == "__main__":
    ui.launch(share=True)  # share=True creates a public link for Colab